In [1]:
import xarray as xr
import geopandas as gpd
import dask
import numpy as np
import pendulum
from dateutil.relativedelta import relativedelta
import warnings
warnings.filterwarnings("ignore") #ignora runtime warning por causa de nan na hora da correlacao

from skill import recorte
from skill import clima
from skill import plot
from skill import config

# Configurando diretorios

In [2]:
dir_dados = config.dir_dados
dir_img = config.dir_img
dir_shp = config.dir_shp

# Abrindo arquivos

In [3]:
gpcc = xr.open_dataset(f'{dir_dados}/precip.comb.v2020to2019-v2020monitorafter.total.nc')
br_madeira = gpd.read_file(f'{dir_shp}/brasil_madeira.shp')

clima_cam = xr.open_dataset(f'{dir_dados}/cam3_clima.nc')
clima_cam['chuva'] = (clima_cam.PRECC + clima_cam.PRECSH + clima_cam.PRECL)*3600*24*30*1000 #cria chuva total com as variaveis do dataset

## Abrindo arquivo de previsão p/ utilizar lat e lon p/ interpolacao da climatologia e do observado

A climatologia tem mesma resolução que as previsões do CAM. Entretanto, os pontos de grade não são os mesmos. Isso acaba causando problemas na hora de calcular a anomalia, já que a indexação é feita pelo conjunto lat lon.

In [4]:
aux = xr.open_dataset(f'{dir_dados}/cam3_init1.nc')

lat = aux.lat.values
lon = aux.lon.values

In [5]:
clima_cam = recorte.regridder(clima_cam, lat, lon)

# Trabalhando com observado

## Calcula anomalia

Climatologia escolhida para ter mesma data da climatologia do CAM de acordo com Vasconcellos (2017).

In [6]:
clima_obs = clima.climatologia(gpcc, '1979-01-01', '1999-12-01')
obs = gpcc.sel(time=slice('2012-01-01', '2021-09-01'))

anom_obs = clima.anomalia(obs, clima_obs)

## Interpola para a grade do cam

In [7]:
anom_obs_interp = recorte.regridder(anom_obs, lat, lon)
anom_obs_br = recorte.recortar(anom_obs_interp, br_madeira)

# Fazendo correlação

In [8]:
init = np.arange(1,13,1)
leads = np.arange(1,4,1)

for i in init:
    cam = xr.open_dataset(f'{dir_dados}/cam3_init{i}.nc')
    cam['chuva'] = (cam.PRECC + cam.PRECSH + cam.PRECL)*3600*24*30*1000


    cam_anom = clima.anomalia(cam.chuva, clima_cam.chuva.groupby('time.month').mean())
    cam_anom_br = recorte.recortar(cam_anom, br_madeira)
  
    
    data = pendulum.now().set(year=1975, month=i, day=21)
    result = list()
    for lead in leads:
        
        mes = (data + relativedelta(months=lead)).month
        
        ds_camcorr = cam_anom_br.sel(time=cam_anom_br.time.dt.month == mes, lead=lead)
        ds_obscorr = anom_obs_br.sel(time=ds_camcorr.time.values).precip
        
        corr = clima.correlacao(ds_camcorr, ds_obscorr)
        result.append(corr)
        
    ds_corr_br = xr.concat(result, dim='lead')
    ds_corr_br.to_netcdf(f'{dir_dados}/cam3-anom_corr-init{i}.nc')